# OpenStreetMap Data Wrangling

### Author: Carl (Andrew) Perkins

### Date: April 9, 2019

# Map Area

### Beechmont (Louisville), KY, U.S.A.

The data used for this case study was pulled from Open Street Map using the Overpass API at the following link:
<a href="https://www.openstreetmap.org/#map=5/51.500/-0.100">Open Street Map</a>

The data was originally imported as an OSM file, converted into CSV and finally pulled into SQLite.

Beechmont is a municipality of the great Louisville Metro area in Kentucky. I currently live in Louisville, and I am interested to see what contributions are currently in the map and what can be known moving forward using a variety of query techniques. 

## Identifying Problems in the Map

#### First I looked at a 1% sample of the 151 MB dataset; I then used a variety of techniques as follows to identify problems in the larger set of data:

#### 1.) First after reviewing some of the data, I used Python code to audit and clean the street addresses within the OSM file itself. Most of the dirty data was due to over abbreviated street names, phone numbers or lengths of the numbers themselves. The Python code used to audit and clean the addresses is located below. The code directly following the below code is the Python used to import into CSV files and clean the data just a bit more.


#### 2.) After I converted the data into CSV and imported into SQLlite, I used the following queries to audit once more for a final pass at any inaccurate "dirty" data.

##### Please note that the following Schema was used in the SQL Database:
<a href="https://gist.github.com/swwelch/f1144229848b407e0a5d13fcb7fbbd6f">Schema</a>

### Python code used to audit and clean streed addresses

In [11]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint
from num2words import num2words

OSMFILE = "map.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
nth_re = re.compile(r'\d\d?(st|nd|rd|th|)', re.IGNORECASE)
nesw_re = re.compile(r'\s(North|East|South|West)$')

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Circle", "Crescent", "Gate", "Terrace", "Grove", "Way"]

mapping = { 
            "St": "Street",
            "St.": "Street",
            "STREET": "Street",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "Dr.": "Drive",
            "Dr": "Drive",
            "Rd": "Road",
            "Rd.": "Road",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
            "Ehs": "EHS",
            "Trl": "Trail",
            "Cir": "Circle",
            "Cir.": "Circle",
            "Ct": "Court",
            "Ct.": "Court",
            "Crt": "Court",
            "Crt.": "Court",
            "By-pass": "Bypass",
            "N.": "North",
            "N": "North",
            "E.": "East",
            "E": "East",
            "S.": "South",
            "S": "South",
            "W.": "West",
            "W": "West"
          }

street_mapping = {
                   "St": "Street",
                   "St.": "Street",
                   "ST": "Street",
                   "STREET": "Street",
                   "Ave": "Avenue",
                   "Ave.": "Avenue",
                   "Rd.": "Road",
                   "Dr.": "Drive",
                   "Dr": "Drive",
                   "Rd": "Road",
                   "Rd.": "Road",
                   "Blvd": "Boulevard",
                   "Blvd.": "Boulevard",
                   "Ehs": "EHS",
                   "Trl": "Trail",
                   "Cir": "Circle",
                   "Cir.": "Circle",
                   "Ct": "Court",
                   "Ct.": "Court",
                   "Crt": "Court",
                   "Crt.": "Court",
                   "By-pass": "Bypass"
                }


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit_street(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types



### Python code used to clean phone number and insert into CSV files

In [15]:
import csv
import codecs
import re
import xml.etree.cElementTree as ET

import cerberus



OSM_PATH = "map.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
PHONENUM = re.compile(r'\+1\s\d{3}\s\d{3}\s\d{4}')
POSTCODE = re.compile(r'[A-z]\d[A-z]\s?\d[A-z]\d')

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']



def update_phone_num(phone_num):
    """
    Clean phone number for insertion into SQL database
    """
    # Check for valid phone number format
    m = PHONENUM.match(phone_num)
    if m is None:
        # Convert all dashes to spaces
        if "-" in phone_num:
            phone_num = re.sub("-", " ", phone_num)
        # Remove all brackets
        if "(" in phone_num or ")" in phone_num:
            phone_num = re.sub("[()]", "", phone_num)
        # Space out 10 straight numbers
        if re.match(r'\d{10}', phone_num) is not None:
            phone_num = phone_num[:3] + " " + phone_num[3:6] + " " + phone_num[6:]
        # Space out 11 straight numbers
        elif re.match(r'\d{11}', phone_num) is not None:
            phone_num = phone_num[:1] + " " + phone_num[1:4] + " " + phone_num[4:7] + " " + phone_num[7:]
        # Add full country code
        if re.match(r'\d{3}\s\d{3}\s\d{4}', phone_num) is not None:
            phone_num = "+1 " + phone_num
        # Ignore tag if no area code and local number (<10 digits)
        elif sum(c.isdigit() for c in phone_num) < 10:
            return None
    return phone_num


def load_new_tag(element, secondary, default_tag_type):
    """
    Load a new tag dict to go into the list of dicts for way_tags, node_tags
    """
    new = {}
    new['id'] = element.attrib['id']
    if ":" not in secondary.attrib['k']:
        new['key'] = secondary.attrib['k']
        new['type'] = default_tag_type
    else:
        post_colon = secondary.attrib['k'].index(":") + 1
        new['key'] = secondary.attrib['k'][post_colon:]
        new['type'] = secondary.attrib['k'][:post_colon - 1]

    return new
def string_case(s):  # change string into titleCase except for UpperCase
    if s.isupper():
        return s
    else:
        return s.title()
    
# return the updated names
def update_name(name, mapping):
    name = name.split(' ')
    for i in range(len(name)):
        if name[i] in mapping:
            name[i] = mapping[name[i]]
            name[i] = string_case(name[i])
        else:
            name[i] = string_case(name[i])

    name = ' '.join(name)

    return name


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []

    if element.tag == 'node':
        for attrib in element.attrib:
            if attrib in NODE_FIELDS:
                node_attribs[attrib] = element.attrib[attrib]

        for child in element:
            node_tag = {}
            if LOWER_COLON.match(child.attrib['k']):
                node_tag['type'] = child.attrib['k'].split(':', 1)[0]
                node_tag['key'] = child.attrib['k'].split(':', 1)[1]
                node_tag['id'] = element.attrib['id']
                node_tag['value'] = update_name(child.attrib['v'],mapping)
                tags.append(node_tag)
            elif PROBLEMCHARS.match(child.attrib['k']):
                continue
            else:                
                node_tag['key'] = child.attrib['k']
                node_tag['type'] = 'regular'
                node_tag['id'] = element.attrib['id']
                node_tag['value'] = update_name(child.attrib['v'],mapping)
                tags.append(node_tag)

        return {'node': node_attribs, 'node_tags': tags}

    elif element.tag == 'way':
        for attrib in element.attrib:
            if attrib in WAY_FIELDS:
                way_attribs[attrib] = element.attrib[attrib]

        position = 0
        for child in element:
            way_tag = {}
            way_node = {}

            if child.tag == 'tag':
                if LOWER_COLON.match(child.attrib['k']):
                    way_tag['type'] = child.attrib['k'].split(':', 1)[0]
                    way_tag['key'] = child.attrib['k'].split(':', 1)[1]
                    way_tag['id'] = element.attrib['id']
                    way_tag['value'] = update_name(child.attrib['v'],mapping)
                    tags.append(way_tag)
                elif PROBLEMCHARS.match(child.attrib['k']):
                    continue
                else:
                    way_tag['type'] = 'regular'
                    way_tag['key'] = child.attrib['k']
                    way_tag['id'] = element.attrib['id']
                    way_tag['value'] = update_name(child.attrib['v'],mapping)
                    tags.append(way_tag)

            elif child.tag == 'nd':
                way_node['id'] = element.attrib['id']
                way_node['node_id'] = child.attrib['ref']
                way_node['position'] = position
                position += 1
                way_nodes.append(way_node)

        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)
            

# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)

### SQL code used to audit data as a final pass

In [3]:
import pandas as pd
import sqlite3
conn = sqlite3.connect(r"Data_Wrangling_DB.db")

#This following code reads is looking for any postal codes that don't match a 5 length format -- the expected outcome should
# be 0 rows returned.
pd.read_sql_query("Select * from nodes_tags where [key] = 'postcode' and length(value) < 5;", conn)

,id,key,value,type


The cell above's expected outcome should be 0 rows.  Due to the formatting of the Zipcode we are now seeing that they are all true 5 length characters.

In [4]:
#The following code prints all County_names -- the expected outcome should be "Jefferson"
pd.read_sql_query("Select * from nodes_tags where [key] = 'county_name';", conn)

,id,key,value,type
0,367811539,county_name,Jefferson,gnis
1,367811877,county_name,Jefferson,gnis
2,3418007532,county_name,Jefferson,gnis
3,3418007533,county_name,Jefferson,gnis


All the county_names above are showing that they reside in Jefferson county, which is correct for the city of Louisville.

In [8]:
#The following code should return values where the city is distinctly (Louisville, Shively, Parkway Village, Audubon Park, 
# Lynnview, Watterson Park or Poplar Hills)
pd.read_sql_query("Select distinct [value] from nodes_tags where [key] = 'city';", conn)

,value
0,Shively
1,Louisville
2,Parkway Village
3,Audubon Park
4,Lynnview
5,Watterson Park
6,Poplar Hills


The code above shows the distinct municipalities that are associated to the area in which Beechmont is within Louisville, KY. 

In [26]:
#Reads any phone number where the length would include the country code or a space to exclude it
pd.read_sql_query("Select distinct [value] from nodes_tags where [key] = 'phone' and length(value) > 12;", conn)

#Reads any phone number where the value doesn't include the dashes for easy readability.
pd.read_sql_query("Select distinct [value] from nodes_tags where [key] = 'phone' and value not like '%-%';", conn)

,value


The above shows the phone number values of any phone numbers that are not normalized, most specifically any with a length more than 12 and those that don't have dashes in their value at all.

In summation, most of the data is now cleansed or normalized to a standard across the board.

### Counts, Statistics and more!

In [28]:
# This query is looking to see which street has the most hits for address data
pd.read_sql_query("SELECT t.value, COUNT(*) as count FROM (SELECT * FROM nodes_tags UNION SELECT * FROM ways_tags) t WHERE t.key = 'street' GROUP BY t.value ORDER BY count DESC;", conn)

,value,count
0,Dixie Highway,797
1,South 3Rd Street,730
2,Southern Parkway,664
3,Taylor Boulevard,648
4,Preston Highway,600
5,South 2Nd Street,438
6,Poplar Level Road,414
7,Manslick Road,384
8,Crums Lane,346
9,South 6Th Street,342


Not surprisingly, Dixie Highway comes in number one for the most address hits. Dixie Highway is highly commercialized and has lots of businesses, as well as several homes that are located more outside of the city.

#### Next let's look at our file sizes and differing counts of specific tables:

#### Number of nodes:

In [4]:
pd.read_sql_query("SELECT COUNT(*) FROM nodes;", conn)

,COUNT(*)
0,620676


#### Number of ways:

In [5]:
pd.read_sql_query("SELECT COUNT(*) FROM ways;", conn)

,COUNT(*)
0,79049


#### Number of unique users:

In [6]:
pd.read_sql_query("SELECT COUNT(DISTINCT(users.uid)) FROM (SELECT uid FROM nodes UNION SELECT uid FROM ways) users;", conn)

,COUNT(DISTINCT(users.uid))
0,252


#### Louisville is a "foody" town. The following is a listing of the most popular restaurants in the area:    

In [8]:
pd.read_sql_query("SELECT t.value, COUNT(*) as num FROM nodes_tags t INNER JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') i ON t.id=i.id WHERE t.key='cuisine' GROUP BY t.value ORDER BY num DESC;", conn)

,value,num


Due to the null values I would say that this data needs more "amenity" data across the board.

### Conclusion

In conclusion I was able to clean most of the data and make sure that the data is normalized across the board.  Due to lack of information that I found when researching, I believe that the greatest room for improvement lies within increased data contributions.  Sunergos Coffee, for instance, is down the street and is not in the data. This is a coffee business that has been around for more than a decade, has won multiple national competitions, and is no where to be seen in this data. I would love to see increased data from more contributors down the road.